In [3]:
import numpy as np
#from dependence import VineCopula

In [4]:
dim = 3
family = np.zeros((dim, dim), dtype=int)
for i in range(1, dim):
    for j in range(i):
        family[i, j] = 1

par1 = np.zeros((dim, dim))
for i in range(1, dim):
    for j in range(i):
        par1[i, j] = 0.5
        
par2 = np.zeros((dim, dim))
for i in range(1, dim):
    for j in range(i):
        par2[i, j] = 0.

In [5]:
structure = np.zeros((dim, dim), dtype=int)
for i in range(1, dim):
    for j in range(i):
        structure[i, j] = i
        
structure

array([[0, 0, 0],
       [1, 0, 0],
       [2, 2, 0]])

In [6]:
import rpy2.rinterface as ri
from rpy2.robjects.packages import importr
from rpy2.robjects.numpy2ri import numpy2ri
VINECOPULA = importr('VineCopula')

In [9]:
r_structure = numpy2ri(structure)
r_family = numpy2ri(family)
r_par = numpy2ri(par1)
r_par2 = numpy2ri(par2)
rvine = VINECOPULA.RVineMatrix(r_structure, r_family, r_par, r_par2)

RRuntimeError: Erreur : 
 In function (Matrix, family = array(0, dim = dim(Matrix)), par = array(NA, dim = dim(Matrix)), par2 = array(NA, dim = dim(Matrix)), names = NULL, check.pars = TRUE) 
{
    args <- preproc(c(as.list(environment()), call = match.call()), check_matrix, check_fammat, check_parmat, check_par2mat)
    list2env(args, environment())
    if (length(names) > 0 & length(names) != dim(Matrix)[1]) 
        stop("Length of the vector 'names' is not correct.")
    sel <- lower.tri(family)
    if (check.pars & (any(family != 0) | any(!is.na(par)))) {
        BiCopCheck(family[sel], par[sel], par2[sel], call = match.call())
    }
    MaxMat <- createMaxMat(Matrix)
    CondDistr <- neededCondDistr(Matrix)
    RVM <- list(Matrix = Matrix, family = family, par = par, par2 = par2, names = names, MaxMat = MaxMat, CondDistr = CondDistr)
    class(RVM) <- "RVineMatrix"
    if (is.CVine(RVM)) {
        RVM$type <- "C-vine"
    }
    else if (is.DVine(RVM)) {
        RVM$type <- "D-vin